In [ ]:
#Below code predict the starting stages using already trained models generated

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout
from keras.layers import Conv1D,Conv2D
from sklearn.preprocessing import MinMaxScaler
from keras.optimizers import Adam
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from keras.layers import Flatten
from keras.layers import Dropout,Activation
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from keras.models import load_model
import keras
import pickle

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
    

df_train=pd.read_csv('/kaggle/input/conways-reverse-game-of-life-2020/train.csv')
output_column_name_list=df_train.iloc[:,1:627].columns

def reshape_input(X):
    return X.reshape(X.shape[0], X.shape[1], X.shape[2], 1)

df_test=pd.read_csv('/kaggle/input/conways-reverse-game-of-life-2020/test.csv')
print(df_test)
df_test=df_test.sort_values(by='delta', ascending=True)
delta_list=df_test['delta'].tolist()
print(df_test)
id_list=df_test['id'].tolist()

np_input_data=df_test.iloc[:,2:].values
print(np_input_data)
complete_input_array=[]

#rearange the data in the form of 2d matrix(25x25)
for i in range(np_input_data.shape[0]):
    
    row_array=[]
    np_input = np.empty((0,25), int)
    column_count=1
    for j in range(np_input_data.shape[1]):
       
        if column_count < 26:
            row_array.append(np_input_data[i][j])
            
            if column_count == 25:
                np_input=np.append(np_input,np.array([np.asarray(row_array)]), axis=0)
                row_array=[]
                column_count=0
        
        column_count=column_count+1
    
    
    
    complete_input_array.append(np_input)

complete_input_array_=reshape_input(np.asarray(complete_input_array))


#Obtaining the output predictions form the trained models according to the delta value.
list_=[]

# for j in range(len(delta_list)):
var_=0
for i in range(complete_input_array_.shape[0]):
    if delta_list[i]!=var_:
        if delta_list[i]==1:
            print('one')
            var_=1
            model= load_model('../input/modelss/delta_1')
        elif delta_list[i]==2:
            print('two')
            var_=2
            model= load_model('../input/modelss/delta_2')
        elif delta_list[i]==3:
            print('three')
            var_=3
            model= load_model('../input/modelss/delta_3')
        elif delta_list[i]==4:
            print('four')
            var_=4
            model= load_model('../input/modelss/delta_4')
        elif delta_list[i]==5:
            print('five')
            var_=5
            model= load_model('../input/modelss/delta_5')
    
    result=model.predict_on_batch(complete_input_array_[i:i+1])
    result = (result > 0.4).astype(int)
    list_.append(result.flatten().reshape(-1,625)[0])
#     print(i)
    
# print(np.asarray(list_))
submission_df = pd.DataFrame(np.asarray(list_),columns=list(output_column_name_list[1:]))
print(submission_df)
# print(submission_df)


df_sample=pd.read_csv('/kaggle/input/conways-reverse-game-of-life-2020/sample_submission.csv')

submission_df.insert(0,'id', id_list)
print(submission_df)

submission_df=submission_df.sort_values(by='id', ascending=True)
submission_df.to_csv('submission.csv',index=False)